# Testing Graph Patterns - Ungraph

Este notebook prueba sistemáticamente todas las funcionalidades del sistema de patrones de grafo:

1. **Value Objects**: NodeDefinition, RelationshipDefinition, GraphPattern
2. **Patrones Predefinidos**: FILE_PAGE_CHUNK_PATTERN
3. **PatternService**: Validación y generación de queries Cypher
4. **Patrones de Búsqueda GraphRAG**: Todos los patrones disponibles
5. **Integración Completa**: Uso end-to-end con datos reales

## Objetivo

Validar que todas las funcionalidades funcionan correctamente y documentar su uso.


In [1]:
def add_src_to_path(path_folder: str):
    ''' 
    Helper function for adding the "path_folder" directory to the path.
    in order to work on notebooks and scripts
    '''
    import sys
    from pathlib import Path

    base_path = Path().resolve()
    for parent in [base_path] + list(base_path.parents):
        candidate = parent / path_folder
        if candidate.exists():
            # Agregar el directorio padre para que sea un paquete Python
            parent_dir = candidate.parent
            if str(parent_dir) not in sys.path:
                sys.path.insert(0, str(parent_dir))
                print(f"Path Folder parent added: {parent_dir}")
            if str(candidate) not in sys.path:
                sys.path.append(str(candidate))
                print(f"Path Folder {path_folder} added: {candidate}")
            return
    print(f"Not found '{path_folder}' folder on the hierarchy of directories")

# Agregar carpetas necesarias al path
add_src_to_path(path_folder="src")
add_src_to_path(path_folder="src/utils")
add_src_to_path(path_folder="src/data")


Path Folder parent added: D:\projects\Ungraph
Path Folder src added: D:\projects\Ungraph\src
Path Folder src/utils added: D:\projects\Ungraph\src\utils
Path Folder src/data added: D:\projects\Ungraph\src\data


In [2]:
# Importar librerías necesarias
import sys
from pathlib import Path
from typing import Dict, Any

# Importar handlers desde utils
from src.utils.handlers import find_in_project

# Importar ungraph
try:
    import ungraph
    print("✅ Ungraph importado como paquete instalado")
except ImportError:
    import src
    ungraph = src
    print("✅ Ungraph importado desde src/ (modo desarrollo)")

print(f"📦 Ungraph version: {ungraph.__version__}")


d:\projects\Ungraph\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Ungraph importado desde src/ (modo desarrollo)
📦 Ungraph version: 0.1.0


## Parte 1: Value Objects - NodeDefinition

Probamos la creación y validación de NodeDefinition.


In [3]:
# Importar Value Objects
from domain.value_objects.graph_pattern import (
    NodeDefinition,
    RelationshipDefinition,
    GraphPattern
)

print("✅ Value Objects importados correctamente")


✅ Value Objects importados correctamente


In [4]:
# Test 1.1: Crear NodeDefinition válido
print("\n📝 Test 1.1: Crear NodeDefinition válido")
print("=" * 60)

file_node = NodeDefinition(
    label="File",
    required_properties={"filename": str},
    optional_properties={"createdAt": int},
    indexes=["filename"]
)

print(f"✅ NodeDefinition creado:")
print(f"   Label: {file_node.label}")
print(f"   Propiedades requeridas: {list(file_node.required_properties.keys())}")
print(f"   Propiedades opcionales: {list(file_node.optional_properties.keys())}")
print(f"   Índices: {file_node.indexes}")



📝 Test 1.1: Crear NodeDefinition válido
✅ NodeDefinition creado:
   Label: File
   Propiedades requeridas: ['filename']
   Propiedades opcionales: ['createdAt']
   Índices: ['filename']


In [5]:
# Test 1.2: Validar que NodeDefinition rechaza labels inválidos
print("\n📝 Test 1.2: Validar formato de labels")
print("=" * 60)

try:
    invalid_node = NodeDefinition(label="file")  # Debe empezar con mayúscula
    print("❌ ERROR: Debería haber fallado")
except ValueError as e:
    print(f"✅ Validación funcionando: {e}")

try:
    invalid_node2 = NodeDefinition(label="File-Name")  # No puede tener guiones
    print("❌ ERROR: Debería haber fallado")
except ValueError as e:
    print(f"✅ Validación funcionando: {e}")



📝 Test 1.2: Validar formato de labels
✅ Validación funcionando: Invalid label format: file. Labels must start with uppercase letter and contain only letters, numbers, and underscores.
✅ Validación funcionando: Invalid label format: File-Name. Labels must start with uppercase letter and contain only letters, numbers, and underscores.


## Parte 2: Value Objects - RelationshipDefinition

Probamos la creación y validación de RelationshipDefinition.


In [6]:
# Test 2.1: Crear RelationshipDefinition válido
print("\n📝 Test 2.1: Crear RelationshipDefinition válido")
print("=" * 60)

contains_rel = RelationshipDefinition(
    from_node="File",
    to_node="Page",
    relationship_type="CONTAINS",
    direction="OUTGOING"
)

print(f"✅ RelationshipDefinition creado:")
print(f"   From: {contains_rel.from_node}")
print(f"   To: {contains_rel.to_node}")
print(f"   Type: {contains_rel.relationship_type}")
print(f"   Direction: {contains_rel.direction}")



📝 Test 2.1: Crear RelationshipDefinition válido
✅ RelationshipDefinition creado:
   From: File
   To: Page
   Type: CONTAINS
   Direction: OUTGOING


## Parte 3: Value Objects - GraphPattern

Probamos la creación y validación de GraphPattern completo.


In [7]:
# Test 3.1: Crear GraphPattern completo
print("\n📝 Test 3.1: Crear GraphPattern completo")
print("=" * 60)

# Crear nodos
file_node = NodeDefinition(
    label="File",
    required_properties={"filename": str},
    indexes=["filename"]
)

page_node = NodeDefinition(
    label="Page",
    required_properties={"filename": str, "page_number": int},
    indexes=["filename"]
)

# Crear relación
contains_rel = RelationshipDefinition(
    from_node="File",
    to_node="Page",
    relationship_type="CONTAINS"
)

# Crear patrón
test_pattern = GraphPattern(
    name="TEST_FILE_PAGE",
    description="Patrón de prueba: File contiene Page",
    node_definitions=[file_node, page_node],
    relationship_definitions=[contains_rel]
)

print(f"✅ GraphPattern creado:")
print(f"   Nombre: {test_pattern.name}")
print(f"   Descripción: {test_pattern.description}")
print(f"   Nodos: {len(test_pattern.node_definitions)}")
print(f"   Relaciones: {len(test_pattern.relationship_definitions)}")



📝 Test 3.1: Crear GraphPattern completo
✅ GraphPattern creado:
   Nombre: TEST_FILE_PAGE
   Descripción: Patrón de prueba: File contiene Page
   Nodos: 2
   Relaciones: 1


## Parte 4: Patrones Predefinidos

Probamos el patrón predefinido FILE_PAGE_CHUNK_PATTERN.


In [8]:
# Test 4.1: Importar y examinar FILE_PAGE_CHUNK_PATTERN
print("\n📝 Test 4.1: Examinar FILE_PAGE_CHUNK_PATTERN")
print("=" * 60)

from domain.value_objects.predefined_patterns import FILE_PAGE_CHUNK_PATTERN

pattern = FILE_PAGE_CHUNK_PATTERN

print(f"✅ Patrón predefinido cargado:")
print(f"   Nombre: {pattern.name}")
print(f"   Descripción: {pattern.description}")
print(f"\n   Nodos ({len(pattern.node_definitions)}):")
for node in pattern.node_definitions:
    print(f"     - {node.label}: {len(node.required_properties)} props requeridas, {len(node.optional_properties)} opcionales")

print(f"\n   Relaciones ({len(pattern.relationship_definitions)}):")
for rel in pattern.relationship_definitions:
    print(f"     - {rel.from_node} -[:{rel.relationship_type}]-> {rel.to_node}")



📝 Test 4.1: Examinar FILE_PAGE_CHUNK_PATTERN
✅ Patrón predefinido cargado:
   Nombre: FILE_PAGE_CHUNK
   Descripción: Patrón básico: File contiene Pages, Pages contienen Chunks. Chunks tienen relaciones NEXT_CHUNK entre consecutivos.

   Nodos (3):
     - File: 1 props requeridas, 1 opcionales
     - Page: 2 props requeridas, 0 opcionales
     - Chunk: 4 props requeridas, 3 opcionales

   Relaciones (3):
     - File -[:CONTAINS]-> Page
     - Page -[:HAS_CHUNK]-> Chunk
     - Chunk -[:NEXT_CHUNK]-> Chunk


In [9]:
# Test 4.2: Validar estructura del patrón predefinido
print("\n📝 Test 4.2: Validar estructura del patrón")
print("=" * 60)

pattern = FILE_PAGE_CHUNK_PATTERN

# Verificar que tiene los 3 nodos esperados
node_labels = {node.label for node in pattern.node_definitions}
expected_nodes = {"File", "Page", "Chunk"}

assert node_labels == expected_nodes, f"Nodos esperados: {expected_nodes}, encontrados: {node_labels}"
print(f"✅ Nodos correctos: {node_labels}")

# Verificar relaciones
rel_types = {rel.relationship_type for rel in pattern.relationship_definitions}
expected_rels = {"CONTAINS", "HAS_CHUNK", "NEXT_CHUNK"}

assert rel_types == expected_rels, f"Relaciones esperadas: {expected_rels}, encontradas: {rel_types}"
print(f"✅ Relaciones correctas: {rel_types}")

# Verificar propiedades del nodo Chunk
chunk_node = next(node for node in pattern.node_definitions if node.label == "Chunk")
required_chunk_props = {"chunk_id", "page_content", "embeddings", "embeddings_dimensions"}
assert set(chunk_node.required_properties.keys()) == required_chunk_props
print(f"✅ Propiedades de Chunk correctas: {list(chunk_node.required_properties.keys())}")



📝 Test 4.2: Validar estructura del patrón
✅ Nodos correctos: {'Chunk', 'File', 'Page'}
✅ Relaciones correctas: {'CONTAINS', 'NEXT_CHUNK', 'HAS_CHUNK'}
✅ Propiedades de Chunk correctas: ['chunk_id', 'page_content', 'embeddings', 'embeddings_dimensions']


## Parte 5: PatternService - Validación

Probamos el servicio PatternService para validar patrones.


In [10]:
# Test 5.1: Crear instancia de PatternService
print("\n📝 Test 5.1: Crear PatternService")
print("=" * 60)

from infrastructure.services.neo4j_pattern_service import Neo4jPatternService

service = Neo4jPatternService(database="neo4j")
print(f"✅ Neo4jPatternService creado")
print(f"   Database: {service.database}")



📝 Test 5.1: Crear PatternService
✅ Neo4jPatternService creado
   Database: neo4j


In [11]:
# Test 5.2: Validar patrón predefinido
print("\n📝 Test 5.2: Validar FILE_PAGE_CHUNK_PATTERN")
print("=" * 60)

is_valid = service.validate_pattern(FILE_PAGE_CHUNK_PATTERN)
print(f"✅ Validación: {'VÁLIDO' if is_valid else 'INVÁLIDO'}")
assert is_valid, "El patrón predefinido debe ser válido"



📝 Test 5.2: Validar FILE_PAGE_CHUNK_PATTERN
✅ Validación: VÁLIDO


## Parte 6: PatternService - Generación de Cypher

Probamos la generación dinámica de queries Cypher.


In [12]:
# Test 6.1: Generar query Cypher para patrón simple
print("\n📝 Test 6.1: Generar query Cypher")
print("=" * 60)

# Crear patrón simple
simple_node = NodeDefinition(
    label="TestNode",
    required_properties={"id": str, "name": str},
    optional_properties={"value": int}
)

simple_pattern = GraphPattern(
    name="SIMPLE",
    description="Simple pattern",
    node_definitions=[simple_node],
    relationship_definitions=[]
)

cypher_query = service.generate_cypher(simple_pattern, "create")

print("✅ Query Cypher generado:")
print(cypher_query)

# Verificar que usa parámetros
assert "$id" in cypher_query, "Debe usar parámetros"
assert "$name" in cypher_query
assert "$value" in cypher_query
print("\n✅ Query usa parámetros correctamente")



📝 Test 6.1: Generar query Cypher
✅ Query Cypher generado:
MERGE (n0:TestNode {id: $id, name: $name})
ON CREATE SET n0.value = $value

✅ Query usa parámetros correctamente


In [13]:
# Test 6.2: Generar query Cypher con relaciones
print("\n📝 Test 6.2: Generar query con relaciones")
print("=" * 60)

# Patrón con dos nodos y una relación
node1 = NodeDefinition(label="Node1", required_properties={"id": str})
node2 = NodeDefinition(label="Node2", required_properties={"id": str})
rel = RelationshipDefinition(
    from_node="Node1",
    to_node="Node2",
    relationship_type="RELATES_TO"
)

pattern_with_rel = GraphPattern(
    name="RELATED",
    description="Two related nodes",
    node_definitions=[node1, node2],
    relationship_definitions=[rel]
)

cypher_query = service.generate_cypher(pattern_with_rel, "create")

print("✅ Query Cypher generado:")
print(cypher_query)

# Verificar que incluye la relación
assert "RELATES_TO" in cypher_query
assert "->" in cypher_query
print("\n✅ Query incluye relaciones correctamente")



📝 Test 6.2: Generar query con relaciones
✅ Query Cypher generado:
MERGE (n0:Node1 {id: $id})
MERGE (n1:Node2 {id: $id})
MERGE (n0)-[:RELATES_TO]->(n1)

✅ Query incluye relaciones correctamente


## Parte 7: Crear Patrón Personalizado

Probamos la creación de patrones personalizados y su uso.


In [14]:
# Test 7.1: Crear patrón personalizado simple
print("\n📝 Test 7.1: Crear patrón personalizado")
print("=" * 60)

# Patrón simple: solo chunks sin File/Page
simple_chunk_node = NodeDefinition(
    label="Chunk",
    required_properties={
        "chunk_id": str,
        "content": str
    },
    indexes=["chunk_id"]
)

simple_chunk_pattern = GraphPattern(
    name="SIMPLE_CHUNK",
    description="Solo chunks, sin estructura File-Page",
    node_definitions=[simple_chunk_node],
    relationship_definitions=[]
)

print(f"✅ Patrón personalizado creado:")
print(f"   Nombre: {simple_chunk_pattern.name}")
print(f"   Nodos: {[n.label for n in simple_chunk_pattern.node_definitions]}")
print(f"   Relaciones: {len(simple_chunk_pattern.relationship_definitions)}")

# Validar el patrón
is_valid = service.validate_pattern(simple_chunk_pattern)
print(f"\n✅ Validación: {'VÁLIDO' if is_valid else 'INVÁLIDO'}")

# Generar query Cypher
cypher = service.generate_cypher(simple_chunk_pattern, "create")
print(f"\n📝 Query Cypher generado:")
print(cypher)



📝 Test 7.1: Crear patrón personalizado
✅ Patrón personalizado creado:
   Nombre: SIMPLE_CHUNK
   Nodos: ['Chunk']
   Relaciones: 0

✅ Validación: VÁLIDO

📝 Query Cypher generado:
MERGE (n0:Chunk {chunk_id: $chunk_id, content: $content})


## Parte 8: Resumen de Tests

Resumen de todas las funcionalidades probadas.


In [15]:
print("\n" + "=" * 60)
print("📊 RESUMEN DE TESTS")
print("=" * 60)

tests_summary = {
    "Value Objects": {
        "NodeDefinition": "✅ Creado y validado",
        "RelationshipDefinition": "✅ Creado y validado",
        "GraphPattern": "✅ Creado y validado"
    },
    "Patrones Predefinidos": {
        "FILE_PAGE_CHUNK_PATTERN": "✅ Cargado y validado"
    },
    "PatternService": {
        "Validación": "✅ Funciona correctamente",
        "Generación Cypher": "✅ Genera queries válidos",
        "Aplicación de patrones": "⏳ Pendiente (requiere Neo4j)"
    },
    "Patrones Personalizados": {
        "Creación": "✅ Funciona correctamente",
        "Validación": "✅ Funciona correctamente",
        "Generación Cypher": "✅ Funciona correctamente"
    }
}

for category, items in tests_summary.items():
    print(f"\n{category}:")
    for item, status in items.items():
        print(f"  - {item}: {status}")

print("\n" + "=" * 60)
print("✅ Fase 1 completada exitosamente")
print("📋 Próximos pasos: Fase 2 - Integración con casos de uso")
print("=" * 60)



📊 RESUMEN DE TESTS

Value Objects:
  - NodeDefinition: ✅ Creado y validado
  - RelationshipDefinition: ✅ Creado y validado
  - GraphPattern: ✅ Creado y validado

Patrones Predefinidos:
  - FILE_PAGE_CHUNK_PATTERN: ✅ Cargado y validado

PatternService:
  - Validación: ✅ Funciona correctamente
  - Generación Cypher: ✅ Genera queries válidos
  - Aplicación de patrones: ⏳ Pendiente (requiere Neo4j)

Patrones Personalizados:
  - Creación: ✅ Funciona correctamente
  - Validación: ✅ Funciona correctamente
  - Generación Cypher: ✅ Funciona correctamente

✅ Fase 1 completada exitosamente
📋 Próximos pasos: Fase 2 - Integración con casos de uso


## Referencias

- [Plan de Patrones de Grafo](../../_PLAN_PATRONES_GRAFO.md)
- [Documentación de Patrones](../../docs/concepts/graph-patterns.md)
- [Documentación de Búsqueda GraphRAG](../../docs/api/search-patterns.md)
- [GraphRAG Pattern Catalog](https://graphrag.com/reference/)
- [Neo4j Cypher Manual](https://neo4j.com/docs/cypher-manual/)
